# Repository

In [6]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")


Mounted at /gdrive


In [7]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'

/gdrive/My Drive/Colab Notebooks/Dedoublement


In [8]:
import os  
REP_PROJET = os.getcwd()
REP_INPUT  = REP_PROJET + '/Input/'
REP_INTERMED  = REP_PROJET + '/Intermed/'
REP_OUTPUT  = REP_PROJET + '/Output/'

# Requirements

In [9]:
import sys
import subprocess
import pkg_resources
requirements = set(open('requirements.txt').read().split())
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = requirements - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip', 'install', *missing], stdout=subprocess.DEVNULL)

# MODULES

In [10]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

# Inputs Discogs

Merge the extraction and aggregate data to artist + title level

In [11]:
df_country_date = pd.read_csv(REP_INPUT +'2022-02-24_extraction_discogs__country__released_date.txt',low_memory=False)
df_nocountry_nodate = pd.read_csv(REP_INPUT +'2022-02-23_extraction_discogs.txt')

In [12]:
# merge dataframes
df_discogs = pd.merge(df_nocountry_nodate, df_country_date, how='left',on = ['master_id', 'artist','title'])

# Standarize name and title by Upcase first letter of strings

df_discogs['artist'] = df_discogs['artist'].str.title().str.strip()
df_discogs['title'] = df_discogs['title'].str.title().str.strip()

In [13]:
df_discogs.count()

master_id           9023024
artist              9023024
title               9023024
nb_releases         9023024
nb_released_date    8941994
released_date       7873035
nb_country          8941994
country             8724396
dtype: int64

In [ ]:
# Running time : 10h 
df_discogs[['master_id','released_date','country']]= df_discogs[['master_id','released_date', 'country']].astype(str)

df = df_discogs.groupby(['artist','title']).\
                agg(
                    {
                      'master_id': lambda x: ','.join(set(x)) ,
                      'released_date': lambda x: ','.join(set(x)),
                      'country': lambda x: ','.join(set(x))
                    }
                   )

In [ ]:
df.reset_index().to_pickle(REP_INTERMED + 'df_discogs_by_artist_tile.sav')
baseline = pd.read_pickle(REP_INTERMED + 'df_discogs_by_artist_tile.sav')

# Inputs CDandLP

Scope : 
+ on articles for which we do not know the correspondence on the discogs (id_release=0) 
+ on articles where a sql query can be used to perfectly match discogs and cdandlp data

In [14]:
import pandas as pd
df = pd.read_excel(REP_INPUT +'2022-02-03_seller_reference_sample100K.xlsx')
df.count()

n_ref                   100000
ref_actif               100000
n_ident                 100000
n_categ1                100000
n_categ2                100000
artiste                  99999
titre                   100000
courtdesc_lg1            55537
courtdesc_lg2            22940
format                  100000
annee                    75839
label                    90428
pressage                 84338
id_release              100000
code_upload_not_null    100000
dtype: int64

In [ ]:
na_count_by_columns = df.isnull().sum(axis = 0).reset_index()
na_count_by_columns.columns =['colonnes', 'count']
na_count_by_columns.to_excel(REP_INTERMED + "nan_values.xlsx")
df.loc[53892,'artiste'] =' '


In [ ]:
# Scope on articles for which we do not know the correspondence on the discogs
df = df[df['id_release'] == 0].reset_index(drop=True)
df = df[['n_ref',	'artiste'	,'titre', 'pressage']]
df['artiste'] = df['artiste'].str.title().str.strip()
df['titre'] = df['titre'].str.title().str.strip()

In [ ]:
# dataframe with rows that match perfecly 

# Don't use inner_join because we need the output indexes to be on the same level as df indexes
temp = pd.merge(
          baseline[['artist', 'title']],
          df, 
          how='right',
          left_on=['artist', 'title'],
          right_on=['artiste'	,'titre']).dropna()

   
temp = temp.drop(['artist', 'title'], axis=1)

In [ ]:
# Retrieve perfect rows 
df_no_match = df[~df.index.isin(temp.index)]
df_no_match.head()

,n_ref,artiste,titre,pressage
0,113543790,Francis Cabrel,Les Chemins De Traverse Canada,canada
1,119486700,Rina Ketty,La Madone Aux Fleurs+3,NaN
2,117183052,Patrick Moraz,Future Memories : Patrick Moraz Live On Tv,67.435 - france
6,120090422,Awen Magic Land,Openland,3353570029122 - France
7,118212768,David Calzado & La Charanga Habana - Pa' Que S...,(Sp. Pressing 1 Cd),NaN


In [ ]:
# Supprimer les artistes et titres dupliquées en ne gardant que les prémiers.

In [ ]:
df_no_match.drop_duplicates(subset=['artiste', 'titre'], keep="first", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df_no_match = df_no_match.reset_index(drop=True)
df_no_match.to_pickle(REP_INTERMED + 'base_cdandlp.sav')

In [ ]:
%reset-f